<a href="https://colab.research.google.com/github/karoldem/dh/blob/master/dh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sympy

class dh:
  def __init__(self,n):
    self.table = []
    for i in range (n):
      self.table.append({'a': sympy.symbols(r'a_' + str(i+1)),
                  'al':sympy.symbols(r'\alpha_' + str(i+1)),
                  'd': sympy.symbols(r'd_' +  str(i+1)),
                  't': sympy.symbols(r'\theta_' +  str(i+1))
                  })

  def dhm(self,i):
    return (sympy.Matrix([[sympy.cos(i['t']), -sympy.sin(i['t'])*sympy.cos(i['al']), sympy.sin(i['t'])*sympy.sin(i['al']), i['a']*sympy.cos(i['t'])],
                          [sympy.sin(i['t']), sympy.cos(i['t'])*sympy.cos(i['al']), -sympy.cos(i['t'])*sympy.sin(i['al']), i['a']*sympy.sin(i['t'])],
                          [0, sympy.sin(i['al']), sympy.cos(i['al']), i['d']],
                          [0,0,0,1]]))
    
  def makematrix (self):
    self.t = sympy.eye(4)
    for i in self.table:
      self.t = self.t*self.dhm(i)
  
  def jacobian (self,x):
    self.x = sympy.Matrix(1,0,[])
    for i in x:
      self.x = sympy.Matrix([i]).col_insert(0,self.x)
      pass
    
    self.placement = self.t * sympy.Matrix([[0,0,0,1]]).T
    self.j = self.placement.T.jacobian( self.x )

    self.j.row_del(3)

    for i in range(len( self.j )):
      self.j[i] = sympy.simplify( self.j[i] )
    
  def print_transorm_matrices(self):
    for i in range (len( self.table )):
      t = self.dhm( self.table[i] )

      for j in range (16):
        t[j] = sympy.simplify(t[j])
      
      print (r'$$')
      print (r'^' + str(i) + r'T_' +str(i+1) +  r' = ')
      print(str(sympy.latex(t)))
      print (r'$$')

    print (r'$$')
    print (r'^0 T_'+ str(len(self.table)) + r' = ')
    print(str(sympy.latex( self.t )))
    print (r'$$')

In [11]:
antropomorphic = dh(3) #constructor creates dh table with special kind of variables (google sympy library for more)

#here we replace those variables with constatns (al means alpha, t means theta)
antropomorphic.table[0]['a'] = 0
antropomorphic.table[0]['al'] = sympy.rad(90)

antropomorphic.table[1]['d'] = 0
antropomorphic.table[1]['al'] = 0

antropomorphic.table[2]['d'] = 0
antropomorphic.table[2]['al'] = 0

#transformation matrix based on table
antropomorphic.makematrix()

# jacobian of placement of TCP zero point based on transformation matrix in function of arguments of function below
antropomorphic.jacobian([antropomorphic.table[0]['t'], 
                         antropomorphic.table[1]['t'], 
                         antropomorphic.table[2]['t'] ])
print(r'$$')
print(str(sympy.latex( antropomorphic.j )))
print(r'$$')
print('')

antropomorphic.print_transorm_matrices()

$$
\left[\begin{matrix}- \left(a_{2} \cos{\left (\theta_2 \right )} + a_{3} \cos{\left (\theta_2 + \theta_3 \right )}\right) \sin{\left (\theta_1 \right )} & - \left(a_{2} \sin{\left (\theta_2 \right )} + a_{3} \sin{\left (\theta_2 + \theta_3 \right )}\right) \cos{\left (\theta_1 \right )} & - a_{3} \sin{\left (\theta_2 + \theta_3 \right )} \cos{\left (\theta_1 \right )}\\\left(a_{2} \cos{\left (\theta_2 \right )} + a_{3} \cos{\left (\theta_2 + \theta_3 \right )}\right) \cos{\left (\theta_1 \right )} & - \left(a_{2} \sin{\left (\theta_2 \right )} + a_{3} \sin{\left (\theta_2 + \theta_3 \right )}\right) \sin{\left (\theta_1 \right )} & - a_{3} \sin{\left (\theta_1 \right )} \sin{\left (\theta_2 + \theta_3 \right )}\\0 & a_{2} \cos{\left (\theta_2 \right )} + a_{3} \cos{\left (\theta_2 + \theta_3 \right )} & a_{3} \cos{\left (\theta_2 + \theta_3 \right )}\end{matrix}\right]
$$

$$
^0T_1 = 
\left[\begin{matrix}\cos{\left (\theta_1 \right )} & 0 & \sin{\left (\theta_1 \right )} & 0\\\sin{\